In [28]:
# Installs (uncomment to install)
#!pip install yfinance matplotlib

In [29]:
# Imports
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

from google.colab import output
import ipywidgets as widgets
from IPython.display import display, clear_output

In [30]:
# Ticker universe
tickers = [

    # Sector ETFs
    'XLY',  # Consumer Discretionary Select Sector SPDR Fund
    'XLP',  # Consumer Staples Select Sector SPDR Fund
    'XLE',  # Energy Select Sector SPDR Fund
    'XLF',  # Financial Select Sector SPDR Fund
    'XLV',  # Health Care Select Sector SPDR Fund
    'XLI',  # Industrial Select Sector SPDR Fund
    'XLB',  # Materials Select Sector SPDR Fund
    'XLRE', # Real Estate Select Sector SPDR Fund
    'XLK',  # Technology Select Sector SPDR Fund
    'XLU',  # Utilities Select Sector SPDR Fund

    # Additional ETFs
    'SCHA', # Schwab U.S. Small-Cap ETF
    'VONG', # Vanguard Russell 1000 Growth ETF
    'IWD',  # iShares Russell 1000 Value ETF
    'IDEV', # iShares International Developed ETF
    'INDA', # iShares MSCI India ETF
    'EWJ',  # iShares MSCI Japan ETF
]

num_days = 20
moving_average_window = 3

In [ ]:
# Function to get daily percentage changes for a specified window
def get_daily_percent_change(tickers, ma_days=3):
    percent_changes = []

    for ticker in tickers:
        # Download historical data for the specified window
        data = yf.download(ticker, period='1mo', interval='1d', progress=False)  # Fetch extra days for MA calculation

        # Calculate the 20-day moving average
        moving_avg = data['Close'].rolling(window=ma_days).mean()

        # Calculate daily percentage changes of the 20-day moving average
        pct_change = moving_avg.pct_change().fillna(0)  # Fill NaN for the first row
        #pct_change = data['Close'].pct_change().fillna(0)
        percent_changes.append(pct_change)

    return np.array(percent_changes)

# Function to get a mixed color based on percentage change
def get_color_from_change(change, min_change, max_change):
    # Clamp the change to be within the specified min and max range
    change = max(min(change, max_change), min_change)

    # Normalize change to a ratio between 0 (fully red) and 1 (fully green)
    ratio = (change - min_change) / (max_change - min_change)

    # Calculate the RGB mix: (R, G, B) where red and green vary based on ratio
    red = int((1 - ratio) * 255)
    green = int(ratio * 255)

    # Return the color in hex format
    return f'#{red:02x}{green:02x}00'

def plot_grid(tickers, percent_changes, days_back):
    day_index = -days_back if days_back <= 20 else -1
    selected_day_changes = percent_changes[:, day_index]

    min_change, max_change = -0.02, 0.02
    colors = [get_color_from_change(change, min_change, max_change) for change in selected_day_changes]

    fig, ax = plt.subplots(4, 4, figsize=(8, 8))

    for i in range(4):
        for j in range(4):
            idx = i * 4 + j
            if idx < len(tickers):
                color = colors[idx]
                ax[i, j].add_patch(plt.Rectangle((0, 0), 1, 1, color=color))
                ax[i, j].text(0.5, 0.5, f"{tickers[idx]}\n{selected_day_changes[idx]:.2%}",
                              color='white', fontsize=10, ha='center', va='center', fontweight='bold')
            else:
                ax[i, j].add_patch(plt.Rectangle((0, 0), 1, 1, color='gray'))
                ax[i, j].text(0.5, 0.5, 'N/A', color='white', fontsize=10, ha='center', va='center', fontweight='bold')
            ax[i, j].axis('off')

    plt.tight_layout()
    return fig

# Fetch data once
percent_changes = get_daily_percent_change(tickers, ma_days=moving_average_window)

# Create widgets
days_back = widgets.IntSlider(min=1, max=20, step=1, value=1, description='Days Back:')
prev_button = widgets.Button(description='Previous')
next_button = widgets.Button(description='Next')

# Display widgets
display(widgets.HBox([prev_button, days_back, next_button]))

# Function to update plot
def update_plot(days):
    clear_output(wait=True)
    display(widgets.HBox([prev_button, days_back, next_button]))
    fig = plot_grid(tickers, percent_changes, days)
    plt.show(fig)

# Button click handlers
def on_button_clicked(b):
    if b.description == 'Previous':
        days_back.value = max(days_back.value - 1, days_back.min)
    else:
        days_back.value = min(days_back.value + 1, days_back.max)

# Connect handlers
days_back.observe(lambda change: update_plot(change.new), names='value')
prev_button.on_click(on_button_clicked)
next_button.on_click(on_button_clicked)

# Initial plot
update_plot(days_back.value)